In [9]:
import openai
import torch
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFDirectoryLoader,UnstructuredPDFLoader
import os
from langchain.embeddings.openai import OpenAIEmbeddings
openai.api_key = os.getenv("OPENAI_API_Key")
from huggingface_hub import login
from langchain.embeddings import HuggingFaceInstructEmbeddings
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [10]:
pdf_folder_path = './dateset1'
os.listdir(pdf_folder_path)

['(ASCE)CO.1943-7862.0001590.pdf', '(ASCE)CO.1943-7862.0001593.pdf']

In [11]:
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]
date = []
for loader in loaders:
    date.extend(loader.load())

In [12]:
login(token = os.getenv("Huggingface"))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\a1403\.cache\huggingface\token
Login successful


In [13]:
embeddings = HuggingFaceInstructEmbeddings(cache_folder = "./embeddings" , model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": DEVICE})

load INSTRUCTOR_Transformer
max_seq_length  512


In [14]:
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=1000)
documents = text_splitter.split_documents(date)
db = Chroma.from_documents(documents, embeddings,persist_directory="./chroma_db")

Created a chunk of size 2138, which is longer than the specified 1000
Created a chunk of size 1214, which is longer than the specified 1000
Created a chunk of size 1499, which is longer than the specified 1000
Created a chunk of size 1019, which is longer than the specified 1000
Created a chunk of size 1004, which is longer than the specified 1000
Created a chunk of size 1031, which is longer than the specified 1000
Created a chunk of size 1563, which is longer than the specified 1000
Created a chunk of size 1334, which is longer than the specified 1000
Created a chunk of size 1169, which is longer than the specified 1000
Created a chunk of size 1367, which is longer than the specified 1000
Created a chunk of size 1009, which is longer than the specified 1000
Created a chunk of size 1178, which is longer than the specified 1000
Created a chunk of size 2501, which is longer than the specified 1000
Created a chunk of size 1090, which is longer than the specified 1000
Created a chunk of s

we are down for date base

In [15]:
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

In [16]:
docs = db3.similarity_search("Occupational Safety and Health Admin")
print(docs[0])

page_content='(cid:129) Safety education and training should consider subgroup differ- ences between management personnel and workers, as well as workers from different trades. It is suggested that while safety policies should be consistently implemented to all site employ- ees, demographic or subgroup factors should also be addressed, especially to those subgroups that tend to perceive a lower degree of danger of safety hazards.\n\n© ASCE\n\n04018117-9\n\nJ. Constr. Eng. Manage.\n\nJ. Constr. Eng. Manage., 2019, 145(1): 04018117\n\nDownloaded from ascelibrary.org by Mississippi State Univ Lib on 07/03/19. Copyright ASCE. For personal use only; all rights reserved.' metadata={'source': './dateset1\\(ASCE)CO.1943-7862.0001590.pdf'}
